<a href="https://colab.research.google.com/github/skotschi/KI-Kreativit-t/blob/main/210210_customModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tokenizers

     |████████████████████████████████| 3.2MB 7.7MB/s 


In [2]:
# Datensatz importieren. Beispielhaft wird ein Satz an Pegida Kommentaren genutzt.
# Ursprüngliche Quelle der Daten ist https://github.com/adbar/German-NLP
file_URL = 'https://raw.githubusercontent.com/skotschi/KI-Kreativit-t/main/pegida_korpus.txt'

from requests import get
with open('pegida_korpus.txt', 'wb') as big_f:
    response = get(file_URL, )
    
    if response.status_code == 200:
        big_f.write(response.content)
    else:
        print("Unable to get the file: {}".format(response.reason))

with open ("pegida_korpus.txt", "r") as myfile:
  data=myfile.readlines()

# Eingegebene Daten bereinigen
# Regex zum Filtern der Einträge
import re

# Die einzelnen Kommentare sind mit 47 # Zeichen voneinander getrennt
rule1 = re.compile("#{47}")
# Datum der jeweiligen Kommentare raus filter. Die Zeile beginnt immer mit Jahreszahl-Monat JJJJ-MM
rule2 = re.compile("\d{4}-\d{2}")

# Ich weiß nicht, ob man die beiden nicht auch kombinieren kann. Also in zwei Zeilen.
filtered_data = [i for i in data if not rule1.match(i)]
filtered_data = [i for i in filtered_data if not rule2.match(i)]


In [3]:
# For the user's convenience `tokenizers` provides some very high-level classes encapsulating
# the overall pipeline for various well-known tokenization algorithm. 
# Everything described below can be replaced by the ByteLevelBPETokenizer class. 

from tokenizers import Tokenizer
from tokenizers.decoders import ByteLevel as ByteLevelDecoder
from tokenizers.models import BPE
from tokenizers.normalizers import Lowercase, NFKC, Sequence
from tokenizers.pre_tokenizers import ByteLevel

# First we create an empty Byte-Pair Encoding model (i.e. not trained model)
tokenizer = Tokenizer(BPE())

# Then we enable lower-casing and unicode-normalization
# The Sequence normalizer allows us to combine multiple Normalizer that will be
# executed in order.
tokenizer.normalizer = Sequence([
    NFKC(),
    Lowercase()
])

# Our tokenizer also needs a pre-tokenizer responsible for converting the input to a ByteLevel representation.
tokenizer.pre_tokenizer = ByteLevel()

# And finally, let's plug a decoder so we can recover from a tokenized input to the original one
tokenizer.decoder = ByteLevelDecoder()

!touch filtered_pegida.txt

with open('filtered_pegida.txt', 'w') as f:
    f.writelines(filtered_data)
f.close()

In [4]:
from tokenizers.trainers import BpeTrainer

# We initialize our trainer, giving him the details about the vocabulary we want to generate
trainer = BpeTrainer(vocab_size=25000, show_progress=True, initial_alphabet=ByteLevel.alphabet())
tokenizer.train(files=["filtered_pegida.txt"], trainer=trainer)

print("Trained vocab size: {}".format(tokenizer.get_vocab_size()))

Trained vocab size: 25000


In [5]:
# You will see the generated files in the output.
!mkdir pegibert
tokenizer.model.save('./pegibert')

['./pegibert/vocab.json', './pegibert/merges.txt']

In [6]:
# Let's tokenizer a simple input
tokenizer.model = BPE('./pegibert/vocab.json', './pegibert/merges.txt')
encoding = tokenizer.encode("Ich wiederhole, aber das ist die Wahrheit. Auch Vollverschleierung ist ein schleichender Prozess. Der Weg zur Dunkelheit passiert nicht von heute auf Morgen. Aber am Ende steht immer die Finsternis.")

print("Encoded string: {}".format(encoding.tokens))

decoded = tokenizer.decode(encoding.ids)
print("Decoded string: {}".format(decoded))

Encoded string: ['Ġich', 'Ġwiederhole', ',', 'Ġaber', 'Ġdas', 'Ġist', 'Ġdie', 'Ġwahrheit', '.', 'Ġauch', 'Ġvoll', 'verschleier', 'ung', 'Ġist', 'Ġein', 'Ġschleichen', 'der', 'Ġprozess', '.', 'Ġder', 'Ġweg', 'Ġzur', 'Ġdunkelheit', 'Ġpassiert', 'Ġnicht', 'Ġvon', 'Ġheute', 'Ġauf', 'Ġmorgen', '.', 'Ġaber', 'Ġam', 'Ġende', 'Ġsteht', 'Ġimmer', 'Ġdie', 'Ġf', 'inst', 'ern', 'is', '.']
Decoded string:  ich wiederhole, aber das ist die wahrheit. auch vollverschleierung ist ein schleichender prozess. der weg zur dunkelheit passiert nicht von heute auf morgen. aber am ende steht immer die finsternis.


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: Deprecated in 0.9.0: BPE.__init__ will not create from files anymore, try `BPE.from_file` instead
  


In [7]:
!pip install transformers

     |████████████████████████████████| 1.8MB 6.9MB/s 
     |████████████████████████████████| 890kB 30.2MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=ef3c90ab8c574017b020097fca3b0f3c0219d191a5c5e90ff8666a569cdf6161
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [8]:
!nvidia-smi

Sat Feb 13 21:03:44 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [9]:
import torch
torch.cuda.is_available()

True

In [10]:
from transformers import RobertaConfig

config = RobertaConfig(
    vocab_size=52_000,
    max_position_embeddings=514,
    num_attention_heads=12,
    num_hidden_layers=6,
    type_vocab_size=1,
)

In [11]:
!ls -la

total 119592
drwxr-xr-x 1 root root     4096 Feb 13 21:03 .
drwxr-xr-x 1 root root     4096 Feb 13 20:49 ..
drwxr-xr-x 1 root root     4096 Feb 10 14:40 .config
-rw-r--r-- 1 root root 50916549 Feb 13 21:02 filtered_pegida.txt
drwxr-xr-x 2 root root     4096 Feb 13 21:03 pegibert
-rw-r--r-- 1 root root 71518149 Feb 13 21:02 pegida_korpus.txt
drwxr-xr-x 1 root root     4096 Feb 10 14:40 sample_data


In [12]:
from transformers import RobertaTokenizerFast

tokenizer = RobertaTokenizerFast.from_pretrained("./pegibert", max_len=512)

Special tokens have been added in the vocabulary, make sure the associated word embedding are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embedding are fine-tuned or trained.


In [13]:
from transformers import RobertaForMaskedLM

model = RobertaForMaskedLM(config=config)

In [14]:
model.num_parameters()

83504416

In [15]:
%%time
from transformers import LineByLineTextDataset

dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path="./pegida_korpus.txt",
    block_size=128,
)

/usr/local/lib/python3.6/dist-packages/transformers/data/datasets/language_modeling.py:128: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/language-modeling/run_mlm.py
  FutureWarning,


CPU times: user 1min 21s, sys: 2.95 s, total: 1min 23s
Wall time: 55.4 s


In [16]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

In [17]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./pegibert",
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_gpu_train_batch_size=64,
    save_steps=10_000,
    save_total_limit=2,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
#    prediction_loss_only=True,
)

In [ ]:
%%time
trainer.train()

Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.


Step,Training Loss


In [ ]:
trainer.save_model("./pegibert")

In [ ]:
from transformers import pipeline

fill_mask = pipeline(
    "fill-mask",
    model="./pegibert",
    tokenizer="./pegibert"
)

In [ ]:
fill_mask("In Dresden <mask>.")
print(fill_mask(f"Der {fill_mask.tokenizer.mask_token} ist eine große Gefahr für Deutschland."))
